## Problem Statement:
 Using the latest data, find out the best pinch hitters inside Power Play (PP) in the IPL so far (min of 20 ings)

------------------------------------------------------------------------------------------------------------------------------------------------

### Keywords: Pinch hitter, Powerplay  .
- Quantifying these words:
    - Pinch Hitter = High SR, Low balls_per_boundary, Decent Runs, Decent Balls Faced
    - Powerplay = 1-6 overs

In [2]:
import math
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.expand_frame_repr', False)


import matplotlib.pyplot as plt

In [3]:
df=pd.read_csv("all_matches.csv")

In [4]:
df.head(2)

,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,bowler,runs_off_bat,extras,wides,noballs,byes,legbyes,penalty,wicket_type,player_dismissed,other_wicket_type,other_player_dismissed
0,335982,2007/08,2008-04-18,M Chinnaswamy Stadium,1,0.1,Kolkata Knight Riders,Royal Challengers Bangalore,SC Ganguly,BB McCullum,P Kumar,0.0,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN
1,335982,2007/08,2008-04-18,M Chinnaswamy Stadium,1,0.2,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,P Kumar,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df.columns

Index(['match_id', 'season', 'start_date', 'venue', 'innings', 'ball',
       'batting_team', 'bowling_team', 'striker', 'non_striker', 'bowler',
       'runs_off_bat', 'extras', 'wides', 'noballs', 'byes', 'legbyes',
       'penalty', 'wicket_type', 'player_dismissed', 'other_wicket_type',
       'other_player_dismissed'],
      dtype='object')

In [6]:
# Rename  'striker' to 'batsman'
df.rename(columns = {'striker':'batsman'},inplace=True)

# Rename 'runs_off_bat' column to 'batsman_runs'
df.rename(columns={'runs_off_bat': 'batsman_runs'}, inplace=True)

In [7]:
def phase(ball):


    if ball <= 5.6:
        return 'Powerplay'
    elif ball <= 14.6:
        return 'Middle'
    else:
        return 'Death'

In [8]:
df['phase'] = df['ball'].apply(lambda x: phase(x))

In [10]:
def phasesOfplay(df, current_phase):

    df = df[df.phase == current_phase]
    df.reset_index(inplace = True, drop = True)

    df['isDot'] = df['batsman_runs'].apply(lambda x: 1 if x == 0 else 0)
    df['isOne'] = df['batsman_runs'].apply(lambda x: 1 if x == 1 else 0)
    df['isTwo'] = df['batsman_runs'].apply(lambda x: 1 if x == 2 else 0)
    df['isThree'] = df['batsman_runs'].apply(lambda x: 1 if x == 3 else 0)
    df['isFour'] = df['batsman_runs'].apply(lambda x: 1 if x == 4 else 0)
    df['isSix'] = df['batsman_runs'].apply(lambda x: 1 if x == 6 else 0)

    runs = pd.DataFrame(df.groupby(['batsman', 'match_id'])['batsman_runs'].sum().reset_index()).groupby(['batsman'])['batsman_runs'].sum().reset_index().rename(columns={'batsman_runs':'runs'})
    innings = pd.DataFrame(df.groupby(['batsman'])['match_id'].apply(lambda x: len(list(np.unique(x)))).reset_index()).rename(columns = {'match_id':'innings'})
    balls = pd.DataFrame(df.groupby(['batsman'])['match_id'].count()).reset_index().rename(columns = {'match_id':'balls'})
    dismissals = pd.DataFrame(df.groupby(['batsman'])['player_dismissed'].count()).reset_index().rename(columns = {'player_dismissed':'dismissals'})

    dots = pd.DataFrame(df.groupby(['batsman'])['isDot'].sum()).reset_index().rename(columns = {'isDot':'dots'})
    ones = pd.DataFrame(df.groupby(['batsman'])['isOne'].sum()).reset_index().rename(columns = {'isOne':'ones'})
    twos = pd.DataFrame(df.groupby(['batsman'])['isTwo'].sum()).reset_index().rename(columns = {'isTwo':'twos'})
    threes = pd.DataFrame(df.groupby(['batsman'])['isThree'].sum()).reset_index().rename(columns = {'isThree':'threes'})
    fours = pd.DataFrame(df.groupby(['batsman'])['isFour'].sum()).reset_index().rename(columns = {'isFour':'fours'})
    sixes = pd.DataFrame(df.groupby(['batsman'])['isSix'].sum()).reset_index().rename(columns = {'isSix':'sixes'})

    df = pd.merge(innings, runs, on = 'batsman').merge(balls, on = 'batsman').merge(dismissals, on = 'batsman').merge(dots, on = 'batsman').merge(ones, on = 'batsman').merge(twos, on = 'batsman').merge(threes, on = 'batsman').merge(fours, on = 'batsman').merge(sixes, on = 'batsman')

    #StrikeRate
    df['SR'] = df.apply(lambda x: 100*(x['runs']/x['balls']), axis = 1)

    #runs per innings
    df['RPI'] = df.apply(lambda x: x['runs']/x['innings'], axis = 1)

    #balls per dismissals
    df['BPD'] = df.apply(lambda x: balls_per_dismissal(x['balls'], x['dismissals']), axis = 1)

    #balls per boundary
    df['BPB'] = df.apply(lambda x: balls_per_boundary(x['balls'], (x['fours'] + x['sixes'])), axis = 1)

    return df

In [12]:
pp_df = phasesOfplay(df, 'Powerplay')
mid_df = phasesOfplay(df, 'Middle')
dth_df = phasesOfplay(df, 'Death')

---------------------------------------------------------------------------------------------------------------------------------------------------------------

### Pair wise comparision matrices using Saaty scale & then arriving at weights for each parameter

---------------------------------------------------------------------------------------------------------------------------------------------------------------

In [15]:
wt_sr, wt_rpi, wt_bpd, wt_bpb = 0.38, 0.25, 0.12, 0.26

In [20]:
# filter by min 5 innings
pp_df = pp_df[pp_df.innings >= 20]
pp_df.reset_index(inplace = True, drop = True)

### TOPSIS Method:

**#(alternatives: minmax scaling, standard scaling):**

----------------------------------------------------------------------------------------------------------------------------------------------------------------

### Calculation - Normalizing all values to same dimensions

---------------------------------------------------------------------------------------------------------------------------------------------------------------

In [21]:
pp_df['calc_SR'] = pp_df['SR'].apply(lambda x: x*x)
pp_df['calc_RPI'] = pp_df['RPI'].apply(lambda x: x*x)
pp_df['calc_BPD'] = pp_df['BPD'].apply(lambda x: x*x)
pp_df['calc_BPB'] = pp_df['BPB'].apply(lambda x: x*x)

sq_sr, sq_rpi, sq_bpd, sq_bpb = np.sqrt(pp_df[['calc_SR','calc_RPI', 'calc_BPD', 'calc_BPB']].sum(axis = 0))

pp_df['calc_SR'] = pp_df['calc_SR'].apply(lambda x: x/sq_sr)
pp_df['calc_RPI'] = pp_df['calc_RPI'].apply(lambda x: x/sq_rpi)
pp_df['calc_BPD'] = pp_df['calc_BPD'].apply(lambda x: x/sq_bpd)
pp_df['calc_BPB'] = pp_df['calc_BPB'].apply(lambda x: x/sq_bpb)

pp_df['calc_SR'] = pp_df['calc_SR'].apply(lambda x: x*wt_sr)
pp_df['calc_RPI'] = pp_df['calc_RPI'].apply(lambda x: x*wt_rpi)
pp_df['calc_BPD'] = pp_df['calc_BPD'].apply(lambda x: x*wt_bpd)
pp_df['calc_BPB'] = pp_df['calc_BPB'].apply(lambda x: x*wt_bpb)

best_sr, worst_sr = max(pp_df['calc_SR']), min(pp_df['calc_SR'])
best_rpi, worst_rpi = max(pp_df['calc_RPI']), min(pp_df['calc_RPI'])
best_bpd, worst_bpd = max(pp_df['calc_BPD']), min(pp_df['calc_BPD'])
best_bpb, worst_bpb = min(pp_df['calc_BPB']), max(pp_df['calc_BPB'])

----------------------------------------------------------------------------------------------------------------------------------------------------------------

###  Calculation - Comparision against the best & worst values .

----------------------------------------------------------------------------------------------------------------------------------------------------------------

In [22]:
pp_df['dev_best_SR'] = pp_df['calc_SR'].apply(lambda x: (x-best_sr)*(x-best_sr))
pp_df['dev_best_RPI'] = pp_df['calc_RPI'].apply(lambda x: (x-best_rpi)*(x-best_rpi))
pp_df['dev_best_BPD'] = pp_df['calc_BPD'].apply(lambda x: (x-best_bpd)*(x-best_bpd))
pp_df['dev_best_BPB'] = pp_df['calc_BPB'].apply(lambda x: (x-best_bpb)*(x-best_bpb))

pp_df['dev_best_sqrt'] = pp_df.apply(lambda x: x['dev_best_SR'] + x['dev_best_RPI'] + x['dev_best_BPD'] + x['dev_best_BPB'], axis = 1)

pp_df['dev_worst_SR'] = pp_df['calc_SR'].apply(lambda x: (x-worst_sr)*(x-worst_sr))
pp_df['dev_worst_RPI'] = pp_df['calc_RPI'].apply(lambda x: (x-worst_rpi)*(x-worst_rpi))
pp_df['dev_worst_BPD'] = pp_df['calc_BPD'].apply(lambda x: (x-worst_bpd)*(x-worst_bpd))
pp_df['dev_worst_BPB'] = pp_df['calc_BPB'].apply(lambda x: (x-worst_bpb)*(x-worst_bpb))

pp_df['dev_worst_sqrt'] = pp_df.apply(lambda x: x['dev_worst_SR'] + x['dev_worst_RPI'] + x['dev_worst_BPD'] + x['dev_worst_BPB'], axis = 1)

----------------------------------------------------------------------------------------------------------------------------------------------------------------

### Final Step - Overall score for each player ( values from 0 - 1)

----------------------------------------------------------------------------------------------------------------------------------------------------------------

In [23]:
pp_df['score'] = pp_df.apply(lambda x: x['dev_worst_sqrt']/(x['dev_worst_sqrt'] + x['dev_best_sqrt']), axis = 1)

In [24]:
pp_df[['batsman', 'score']].head()

,batsman,score
0,AB de Villiers,0.010438
1,AC Gilchrist,0.875328
2,BB McCullum,0.398124
3,DA Warner,0.835434
4,G Gambhir,0.420369


---------------------------------------------------------------------------------------------------------------------------------------------------------------

### End Result: The best pinch hitters inside Power Play (PP) in the IPL so far (min of 20 ings).

---------------------------------------------------------------------------------------------------------------------------------------------------------------

In [25]:
pp_df[[ 'batsman','innings', 'runs', 'balls', 'dismissals', 'fours', 'sixes', 'SR', 'BPB', 'score']].sort_values(['score'], ascending = False).reset_index(drop = True).head(25)

,batsman,innings,runs,balls,dismissals,fours,sixes,SR,BPB,score
0,V Sehwag,41,650.0,434,25,93,22,149.769585,3.773913,0.938001
1,AC Gilchrist,49,942.0,673,32,125,41,139.970282,4.054217,0.875328
2,SK Raina,34,404.0,293,9,55,14,137.883959,4.246377,0.860286
3,DA Warner,21,351.0,255,13,44,13,137.647059,4.473684,0.835434
4,KC Sangakkara,30,328.0,255,9,53,5,128.627451,4.396552,0.636998
5,M Vijay,20,346.0,273,9,34,14,126.739927,5.687500,0.599926
6,ST Jayasuriya,30,508.0,401,20,61,26,126.683292,4.609195,0.577729
7,ML Hayden,32,598.0,478,14,81,20,125.104603,4.732673,0.565571
8,G Gambhir,41,660.0,546,19,92,8,120.879121,5.460000,0.420369
9,BB McCullum,23,320.0,265,14,37,16,120.754717,5.000000,0.398124


In [32]:
import plotly.graph_objects as go

# Extracting top pinch hitters dataframe
top_pinch_hitters = pp_df[['batsman', 'score']].sort_values('score', ascending=False).head(25)

# Create diverging colorscale
colorscale = [[0, 'blue'], [0.5, 'lightblue'], [0.5, 'red'], [1, 'green']]

# Create bar plot
fig = go.Figure(data=[go.Bar(
    x=top_pinch_hitters['batsman'],
    y=top_pinch_hitters['score'],
    marker=dict(color=top_pinch_hitters['score'], coloraxis="coloraxis"),
    text=top_pinch_hitters['score'],
    textposition='inside',
)])

# Update layout
fig.update_layout(
    title=dict(text='Top Pinch Hitters Inside Power Play in IPL', font=dict(color='orange')),
    xaxis_title=dict(text='Batsman', font=dict(color='orange', size=16)),
    yaxis_title=dict(text='Score', font=dict(color='orange', size=16)),
    xaxis=dict(tickangle=45, tickfont=dict(color='red', size=12)),
    yaxis=dict(tickformat=".2f", tickfont=dict(color='orange', size=12)),
    coloraxis=dict(colorscale=colorscale),
    plot_bgcolor='rgba(0,0,0,0)',
    paper_bgcolor='rgba(0,0,0,0)',
    font=dict(color='black', size=12),
)

# Show the plot
fig.show()


--------------------------------------------------------------------------------

#The top three pinch hitters in IPL Power Play (min. 20 innings) displayed remarkable batting prowess:

--------------------------------------------------------------------------
1. **Virender Sehwag**: A force to reckon with, Sehwag showcased his aggressive style with a striking 149.77 Strike Rate, scoring an impressive 650 runs.

2. **Adam Gilchrist**: A true dynamo, Gilchrist's explosive batting yielded a whopping 942 runs, setting the field ablaze with a formidable Strike Rate of 139.97.

3. **Suresh Raina**: A consistent performer, Raina's 404 runs at a commendable Strike Rate of 137.88 added depth to his team's lineup, solidifying his reputation as a reliable pinch hitter.

-------------------------------------------------------------------------------
